In [1]:
import pandas as pd
import numpy as np
import os
import sys
import re

import warnings
warnings.filterwarnings('ignore')

In [8]:
cosmic_GRCh37 = pd.read_table("../raw_data/CosmicCompleteGeneExpression_GRCh37.tsv", sep = '\t')

In [10]:
ens69_gtf = pd.read_table("../raw_data/Homo_sapiens.GRCh37.69.gtf", header = None, sep = '\t')

In [11]:
cosmic_GRCh37_median = cosmic_GRCh37.groupby('GENE_NAME')[['GENE_NAME', 'Z_SCORE']].median()

In [13]:
cosmic_GRCh37_median['GENE_NAME'] = cosmic_GRCh37_median.index

In [15]:
cosmic_GRCh37_median.to_csv('../tables_output/cosmic_tcga_median_Z_SCORE.tsv', sep = '\t', index = False)

In [42]:
cosmic_GRCh37_iqr = cosmic_GRCh37.groupby('GENE_NAME')[['GENE_NAME', 'Z_SCORE']].quantile([.25, .5, .75]).unstack()

In [44]:
cosmic_GRCh37_iqr['GENE_NAME'] = cosmic_GRCh37_iqr.index

In [46]:
cosmic_GRCh37_iqr.columns

MultiIndex(levels=[['Z_SCORE', 'GENE_NAME'], [0.25, 0.5, 0.75, '']],
           labels=[[0, 0, 0, 1], [0, 1, 2, 3]])

In [47]:
cols = ['Q1', 'Q2', 'Q3', 'gene_name']

In [50]:
cosmic_GRCh37_iqr.columns = cols
cosmic_GRCh37_iqr = cosmic_GRCh37_iqr.reset_index(drop = True)

In [51]:
cosmic_GRCh37_iqr.head(2)

,Q1,Q2,Q3,gene_name
0,-0.78100,-0.242,0.414,39340
1,-0.60125,-0.283,0.242,A1BG


In [55]:
cosmic_GRCh37_iqr['IQR'] = cosmic_GRCh37_iqr['Q3'] - cosmic_GRCh37_iqr['Q1']
cosmic_GRCh37_iqr['lower_1.5IQR'] = cosmic_GRCh37_iqr['Q1'] - 1.5*cosmic_GRCh37_iqr['IQR']
cosmic_GRCh37_iqr['upper_1.5IQR'] = cosmic_GRCh37_iqr['Q3'] + 1.5*cosmic_GRCh37_iqr['IQR']

In [56]:
cosmic_GRCh37_iqr.to_csv('../tables_output/cosmic_GRCh37_iqr.tsv', sep = '\t', index = False)

---

In [16]:
cosmic_GRCh37_summary = cosmic_GRCh37.groupby('GENE_NAME')[['GENE_NAME', 'Z_SCORE']].describe()

In [19]:
cosmic_GRCh37_summary['gene_name'] = cosmic_GRCh37_summary.index

In [22]:
cosmic_GRCh37_summary = cosmic_GRCh37_summary.reset_index(drop = True)

In [35]:
cosmic_GRCh37_summary.columns.droplevel()

Index(['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', ''], dtype='object')

In [36]:
cols= ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'gene_name']

In [38]:
cosmic_GRCh37_summary.columns

MultiIndex(levels=[['Z_SCORE', 'gene_name'], ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', '']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 1, 2, 3, 4, 5, 6, 7, 8]])

In [39]:
cosmic_GRCh37_summary.columns = cols

In [40]:
cosmic_GRCh37_summary.head(2)

,count,mean,std,min,25%,50%,75%,max,gene_name
0,9144.0,-0.101561,1.197185,-6.089,-0.78100,-0.242,0.414,13.453,39340
1,9144.0,0.334597,25.622720,-1.570,-0.60125,-0.283,0.242,2444.419,A1BG


In [41]:
cosmic_GRCh37_summary.to_csv('../tables_output/cosmic_tcga_GRCh37_summary_stats.tsv', sep = '\t', index = False)

In [60]:
cosmic_GRCh37_summary.shape

(18236, 9)

In [57]:
cosmic_GRCh37_summary_full = pd.merge(cosmic_GRCh37_summary,cosmic_GRCh37_iqr, how = 'inner') 

In [59]:
cosmic_GRCh37_summary_full.shape

(18236, 15)

In [61]:
cosmic_GRCh37_summary_full.to_csv('../tables_output/cosmic_tcga_GRCh37_summary_stats_full.tsv', sep = '\t', index = False)

In [70]:
cosmic_GRCh37_summary_full.head(2)

,count,mean,std,min,25%,50%,75%,max,gene_name,Q1,Q2,Q3,IQR,lower_1.5IQR,upper_1.5IQR
0,9144.0,-0.101561,1.197185,-6.089,-0.78100,-0.242,0.414,13.453,39340,-0.78100,-0.242,0.414,1.19500,-2.573500,2.206500
1,9144.0,0.334597,25.622720,-1.570,-0.60125,-0.283,0.242,2444.419,A1BG,-0.60125,-0.283,0.242,0.84325,-1.866125,1.506875


## Transform Z_score so that they are positive 

In [71]:
min_median_Z_SCORE = min(cosmic_GRCh37_summary_full['Q2'])
min_median_Z_SCORE

-1.1345

In [73]:
cosmic_GRCh37_summary_full['Q2_Z_positive'] = cosmic_GRCh37_summary_full['Q2'] - (min_median_Z_SCORE)

In [75]:
# Get Z_SCORE and normalize to it 
total_median_Z_SCORE = sum(cosmic_GRCh37_summary_full['Q2_Z_positive'])
total_median_Z_SCORE

16883.783499999998

In [76]:
cosmic_GRCh37_summary_full['proportion_Z_score'] = cosmic_GRCh37_summary_full['Q2_Z_positive'] / total_median_Z_SCORE

In [79]:
cosmic_GRCh37_summary_full.head(10)

,count,mean,std,min,25%,50%,75%,max,gene_name,Q1,Q2,Q3,IQR,lower_1.5IQR,upper_1.5IQR,Q2_Z_positive,proportion_Z_score
0,9144.0,-0.101561,1.197185,-6.089,-0.78100,-0.2420,0.41400,13.453,39340,-0.78100,-0.2420,0.41400,1.19500,-2.573500,2.206500,0.8925,0.000053
1,9144.0,0.334597,25.622720,-1.570,-0.60125,-0.2830,0.24200,2444.419,A1BG,-0.60125,-0.2830,0.24200,0.84325,-1.866125,1.506875,0.8515,0.000050
2,9144.0,0.201225,6.548947,-2.224,-0.26100,-0.2030,-0.09600,332.616,A1CF,-0.26100,-0.2030,-0.09600,0.16500,-0.508500,0.151500,0.9315,0.000055
3,9144.0,-0.078867,1.191187,-2.880,-0.76200,-0.3050,0.34225,33.196,A2LD1,-0.76200,-0.3050,0.34225,1.10425,-2.418375,1.998625,0.8295,0.000049
4,9144.0,-0.065317,1.012096,-2.209,-0.61900,-0.3155,0.20700,16.411,A2M,-0.61900,-0.3155,0.20700,0.82600,-1.858000,1.446000,0.8190,0.000049
5,9144.0,0.187292,13.502958,-0.960,-0.33700,-0.2050,-0.08500,1283.790,A2ML1,-0.33700,-0.2050,-0.08500,0.25200,-0.715000,0.293000,0.9295,0.000055
6,9144.0,-0.044697,1.109348,-2.423,-0.69200,-0.3270,0.27600,15.839,A4GALT,-0.69200,-0.3270,0.27600,0.96800,-2.144000,1.728000,0.8075,0.000048
7,9144.0,0.104914,5.252410,-0.884,-0.56400,-0.2525,0.15200,481.559,A4GNT,-0.56400,-0.2525,0.15200,0.71600,-1.638000,1.226000,0.8820,0.000052
8,9144.0,0.088571,1.214318,-3.357,-0.69625,-0.1115,0.65225,16.306,AAAS,-0.69625,-0.1115,0.65225,1.34850,-2.719000,2.675000,1.0230,0.000061
9,9144.0,0.012819,1.168298,-2.765,-0.67425,-0.2280,0.41000,25.542,AACS,-0.67425,-0.2280,0.41000,1.08425,-2.300625,2.036375,0.9065,0.000054


In [78]:
cosmic_GRCh37_summary_full.to_csv('../tables_output/cosmic_tcga_GRCh37_summary_full_median_proportion.tsv', sep = '\t', index = False)

In [80]:
sum(cosmic_GRCh37_summary_full['proportion_Z_score'])

1.0000000000000029